countVectorize 변수 이리저리 적용
1. ngram_range (1,6)

In [0]:
import pandas as pd

In [0]:
#tap으로 구분. seperate. index_col 없으면 판다가 0,1,2...를 만듦
train= pd.read_csv("data/train.tsv", sep="\t", index_col="PhraseId")
test= pd.read_csv("data/test.tsv", sep="\t", index_col="PhraseId")

In [0]:
#sentiment 평점. 4점 높
print(train.shape)
train.head(10)

(156060, 3)


,SentenceId,Phrase,Sentiment
PhraseId,,,
1,1,A series of escapades demonstrating the adage ...,1
2,1,A series of escapades demonstrating the adage ...,2
3,1,A series,2
4,1,A,2
5,1,series,2
6,1,of escapades demonstrating the adage that what...,2
7,1,of,2
8,1,escapades demonstrating the adage that what is...,2
9,1,escapades,2


In [0]:
print(test.shape)
test.head(10)

(66292, 2)


,SentenceId,Phrase
PhraseId,,
156061,8545,An intermittently pleasing but mostly routine ...
156062,8545,An intermittently pleasing but mostly routine ...
156063,8545,An
156064,8545,intermittently pleasing but mostly routine effort
156065,8545,intermittently pleasing but mostly routine
156066,8545,intermittently pleasing but
156067,8545,intermittently pleasing
156068,8545,intermittently
156069,8545,pleasing


## Preprocessing
#### Vectorize Phrases

### Clean Text

In [0]:
# copy하면 원본유지
train["Phrase(Origin)"]= train["Phrase"].copy()

print(train.shape)
train[["Phrase", "Phrase(Origin)"]].head()

(156060, 4)


,Phrase,Phrase(Origin)
PhraseId,,
1,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
2,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
3,A series,A series
4,A,A
5,series,series


In [0]:
def clean_text(phrase):
    phrase= phrase.replace("ca n't","can not")
    phrase= phrase.replace("does n't","does not")    
    phrase= phrase.replace("n't","not")    
    
    return phrase
# ca n't는 오타가 아니라 전처리. can't는 없음

In [0]:
# apply: 반복문처럼 동작
train["Phrase"]= train["Phrase"].apply(clean_text)

print(train.shape)
train[["Phrase", "Phrase(Origin)"]].head()

(156060, 4)


,Phrase,Phrase(Origin)
PhraseId,,
1,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
2,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
3,A series,A series
4,A,A
5,series,series


In [0]:
test["Phrase(Origin)"]= test["Phrase"].copy()

print(test.shape)
test[["Phrase", "Phrase(Origin)"]].head()

(66292, 3)


,Phrase,Phrase(Origin)
PhraseId,,
156061,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156062,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156063,An,An
156064,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156065,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


### Vectorize Phrases !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
# decode_error= 'replace' 변화없음
# strip_accents ascii, unicode (변화 X)
# analyzer char_wb(떨어짐)- fit, transform 오래걸림
# ngram_range (1,6)
# lowercase False(변화 X)
# max_feature=10000: 상위 1만개만 남기겠다

In [0]:
vectorizer= CountVectorizer(ngram_range=(1,6), max_features=10000)
vectorizer
# stop_word: 빈번하게 발생하는 단어. None_디폴트 체크리스트. 따로 넣어줄수도 있음- 한글처리 할때

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 6), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [0]:
#fit / transform
# fit: 빈도수를 기준으로 뭐가 상위인지 결정
vectorizer.fit(train["Phrase"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 6), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [0]:
X_train= vectorizer.transform(train["Phrase"])
X_train

<156060x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 1169513 stored elements in Compressed Sparse Row format>

In [0]:
# get_feature_names: 
vocabulary= vectorizer.get_feature_names()
# vocabulary.remove()
# vocabulary.append()
# 보기좋게 만들기. 이 단어는 요 단어가 몇 번 나왔는지
# 0:1000 - 1천개까지만 보자
pd.DataFrame(X_train[0:1000].toarray(), columns= vocabulary).head()

,10,10 minutes,10 year,100,101,11,12,12 year,12 year old,13,...,your time,your watch,yourself,youth,youthful,zeal,zero,zhang,zombie,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
X_test= vectorizer.transform(test["Phrase"])
X_test

NameError: ignored

In [0]:
y_train= train["Sentiment"]

print(y_train.shape)
y_train.head()

(156060,)


PhraseId
1    1
2    2
3    2
4    2
5    2
Name: Sentiment, dtype: int64

### Train
경험상 여기선 train보단 regression model이 좋더라

In [0]:
from sklearn.linear_model import SGDClassifier

In [0]:
# SGDClassifier: 엄청 많이 쓰는거. 결과가 랜덤하게 나옴. random_state로 고정_seed number
model= SGDClassifier(random_state= 37)
model

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

## N-Gram
나는 딥러닝이 좋습니다> 나는 딥러닝이, 딥러닝이 좋습니다, 나는 딥러닝이 좋습니다. 2개면 2, 3개면 3

In [0]:
# result[result["Phrase"].str.contains("ca n't recommend")]
# "n't recommend" in vocabulary
"stunning" in vocabulary

True

### Scoring

In [0]:
import numpy as np
from sklearn.model_selection import cross_val_predict

In [0]:
y_predict= cross_val_predict(model, X_train, y_train, cv=5)
print(y_predict.shape)
y_predict

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\

(156060,)


array([1, 2, 2, ..., 2, 2, 2], dtype=int64)

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
# 주피터 일괄실행해도, 이 셀은 한번더 실행해야 바뀐 값이 나옴
score= accuracy_score(y_train, y_predict)
print("Score= {0:0.6f}".format(score))

Score= 0.576945


#### countVector 1천개: 0.557061
#### 1만개: 0.573267
#### 1만개+ N-Gram(1,2): 0.574907
#### +Clean Text_***not: 0.576214
#### N-Gram(1,6): 0.576945
### 여기까지해서 제출: 0.59136

### ngram_range
##### (1,6): 0.576945
(1,3): 0.576605, (1,4):0.576317 (1,5):0.575920 (1,7):0.576688 (1,8):0.576016 (1,9):0.575881 (1,10): 0.575971
(2,6): 너무떨어짐

In [0]:
result= train.copy()
result["Sentiment(predict)"]= y_predict
result["Distance"]= result["Sentiment"]- result["Sentiment(predict)"]
result["Distance"]= np.abs(result["Distance"])

# 차이가 큰 순서대로 정렬(많이 틀린거)
result= result.sort_values(by= "Distance", ascending= False)
result.head()

,SentenceId,Phrase,Sentiment,Phrase(Origin),Sentiment(predict),Distance
PhraseId,,,,,,
38914,1856,Men in Black II achieves ultimate insignifican...,0,Men in Black II achieves ultimate insignifican...,4,4
56607,2843,'s never a dull moment in the giant spider inv...,4,'s never a dull moment in the giant spider inv...,0,4
11779,507,is the opposite of a truly magical movie .,0,is the opposite of a truly magical movie .,4,4
14288,616,is an insultingly inept and artificial examina...,0,is an insultingly inept and artificial examina...,4,4
154555,8450,"This method almost never fails him , and it wo...",4,"This method almost never fails him , and it wo...",0,4


In [0]:
result.loc[132656]

SentenceId                                                         7153
Phrase                Miyazaki 's nonstop images are so stunning , a...
Sentiment                                                             4
Phrase(Origin)        Miyazaki 's nonstop images are so stunning , a...
Sentiment(predict)                                                    4
Distance                                                              0
Name: 132656, dtype: object

#### 저장하고 엑셀로

In [0]:
result[0:10000].to_csv("result.csv")

In [0]:
# 분석한 voca랑 뽑아낸 단어의 voca. 2개가 됨
pd.DataFrame(vocabulary).to_csv("vocabulary.csv")

### Predict

In [0]:
model.fit(X_train, y_train)

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [0]:
predictions= model.predict(X_test)

In [0]:
print(predictions.shape)
predictions

(66292,)


array([3, 3, 2, ..., 2, 2, 2], dtype=int64)

### Submit

In [0]:
submission= pd.read_csv("data/sampleSubmission.csv", index_col="PhraseId")

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,2
156062,2
156063,2
156064,2
156065,2


In [0]:
submission["Sentiment"]= predictions

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,3
156062,3
156063,2
156064,3
156065,3


#### 내보내기

In [0]:
submission.to_csv("baseline-script_my1_CVec.csv")